In [13]:
import numpy as np
import pandas as pd 
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.data import Data
import networkx as nx
import torch.nn as nn
from torch_geometric.utils import from_scipy_sparse_matrix, train_test_split_edges
from scipy.sparse import coo_matrix
from sklearn.preprocessing import StandardScaler
from torch_geometric.utils import add_self_loops


In [14]:
#pip install torch torchvision torchaudio
#pip install torch-geometric

In [15]:
ppi_df = pd.read_csv('9606.protein.links.v12.0.txt', delimiter= ' ')
feature_df = pd.read_csv('PPI_feature_mat.csv')
#ppi_df = pd.read_csv('filtered_PPI.csv')


In [16]:
ppi_df

,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000356607,173
1,9606.ENSP00000000233,9606.ENSP00000427567,154
2,9606.ENSP00000000233,9606.ENSP00000253413,151
3,9606.ENSP00000000233,9606.ENSP00000493357,471
4,9606.ENSP00000000233,9606.ENSP00000324127,201
...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,195
13715400,9606.ENSP00000501317,9606.ENSP00000370447,158
13715401,9606.ENSP00000501317,9606.ENSP00000312272,226
13715402,9606.ENSP00000501317,9606.ENSP00000402092,169


In [17]:
feature_df

,Protein,adipose tissue,adrenal gland,amygdala,basal ganglia,breast,cerebellum,cerebral cortex,cervix,colon,...,skeletal muscle,skin,small intestine,spinal cord,spleen,stomach,testis,thyroid gland,urinary bladder,vagina
0,ENSP00000362111,28.7,15.2,7.2,7.8,32.5,2.7,5.7,30.4,31.7,...,2.1,9.2,16.3,6.0,8.4,9.4,97.1,18.9,26.3,24.7
1,ENSP00000362122,20.8,0.0,0.1,0.0,11.7,0.0,0.1,0.1,0.5,...,1.6,3.1,0.4,0.0,0.1,0.1,0.1,0.2,0.2,0.2
2,ENSP00000360638,33.4,38.2,12.4,15.0,33.6,24.3,21.4,35.6,31.1,...,26.3,30.0,29.7,15.5,33.6,25.4,42.5,34.4,32.8,34.6
3,ENSP00000356746,5.5,4.3,1.5,1.7,6.7,5.2,2.3,7.8,5.5,...,3.6,6.9,5.5,2.3,5.8,3.9,6.5,7.3,7.8,6.8
4,ENSP00000286031,1.8,0.9,0.4,0.4,1.7,3.4,0.6,2.1,1.4,...,0.4,1.7,1.5,1.0,1.7,0.9,8.5,1.6,1.6,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19076,ENSP00000479586,0.5,0.8,0.7,1.0,0.5,3.8,1.5,2.2,6.7,...,0.3,2.4,1.0,0.7,0.2,2.0,0.8,0.4,3.9,2.5
19077,ENSP00000492798,0.1,0.1,0.6,0.7,0.1,0.6,0.6,0.2,0.1,...,0.0,0.0,0.1,1.4,0.1,0.0,1.5,0.3,0.1,0.1
19078,ENSP00000491046,0.7,0.4,0.1,0.3,1.1,0.4,0.1,2.3,1.8,...,0.5,2.9,0.6,0.2,1.1,0.9,19.6,2.2,1.5,2.6
19079,ENSP00000491768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def find_common_prefix(strings):
    if not strings:
        return ""
    prefix = strings[0]
    for string in strings[1:]:
        while not string.startswith(prefix):
            prefix = prefix[:-1]
            if not prefix:
                return ""
    return prefix


protein1_prefix = find_common_prefix(ppi_df['protein1'].tolist())
protein2_prefix = find_common_prefix(ppi_df['protein2'].tolist())

print("Common prefix in 'protein1':", protein1_prefix)
print("Common prefix in 'protein2':", protein2_prefix)

proteinf_prefix = find_common_prefix(feature_df['Protein'].tolist())

print("Common prefix in 'feature proteins':", proteinf_prefix)

Common prefix in 'protein1': 9606.ENSP00000
Common prefix in 'protein2': 9606.ENSP00000
Common prefix in 'feature proteins': ENSP00000


In [19]:
ppi_df['protein1'] = ppi_df['protein1'].str.replace(protein1_prefix, '', regex=False)
ppi_df['protein2'] = ppi_df['protein2'].str.replace(protein1_prefix, '', regex=False)
feature_df['Protein'] = feature_df['Protein'].str.replace(proteinf_prefix, '', regex=False)


In [20]:
valid_nodes = set(feature_df['Protein'])
ppi_df_filtered = ppi_df[ppi_df['protein1'].isin(valid_nodes) & ppi_df['protein2'].isin(valid_nodes)]

filtered_nodes = pd.concat([ppi_df_filtered['protein1'], ppi_df_filtered['protein2']]).unique()
node_to_idx = {node: i for i, node in enumerate(filtered_nodes)}

In [ ]:
ppi_df_filtered['protein1_idx'] = ppi_df_filtered['protein1'].map(node_to_idx)
ppi_df_filtered['protein2_idx'] = ppi_df_filtered['protein2'].map(node_to_idx)

C:\Users\siddu\AppData\Local\Temp\ipykernel_52448\3634080600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppi_df_filtered['protein1_idx'] = ppi_df_filtered['protein1'].map(node_to_idx)
C:\Users\siddu\AppData\Local\Temp\ipykernel_52448\3634080600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppi_df_filtered['protein2_idx'] = ppi_df_filtered['protein2'].map(node_to_idx)


In [22]:
adj_matrix = coo_matrix(
    (ppi_df_filtered['combined_score'], (ppi_df_filtered['protein1_idx'], ppi_df_filtered['protein2_idx'])),
    shape=(len(filtered_nodes), len(filtered_nodes))
)

In [23]:
feature_df['node_idx'] = feature_df['Protein'].map(node_to_idx)
feature_df_filtered = feature_df.dropna(subset=['node_idx']).sort_values(by='node_idx')
feature_matrix = feature_df_filtered.iloc[:, 1:-1].values  # exclude 'Protein' and 'node_idx'
feature_matrix = StandardScaler().fit_transform(feature_matrix)

assert adj_matrix.shape[0] == feature_matrix.shape[0]

In [24]:
edge_index, edge_attr = from_scipy_sparse_matrix(adj_matrix)
data = Data(edge_index=edge_index, edge_attr=edge_attr, x=torch.tensor(feature_matrix, dtype=torch.float))

data = train_test_split_edges(data, val_ratio=0.2, test_ratio=0.2)

data.train_pos_edge_index, _ = add_self_loops(data.train_pos_edge_index)


c:\Users\siddu\anaconda3\envs\pytorch_audio\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [25]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        mu = self.conv_mu(x, edge_index)
        logstd = self.conv_logstd(x, edge_index)
        return mu, logstd

In [26]:
in_channels = data.x.shape[1]  # Number of features
out_channels = 32
model = VGAE(GCNEncoder(in_channels, out_channels))

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


In [27]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index)
    loss = model.recon_loss(z, data.train_pos_edge_index)
    loss += (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.train_pos_edge_index)
        auc, ap = model.test(z, data.test_pos_edge_index, data.test_neg_edge_index)
    return auc, ap

In [28]:
for epoch in range(1, 21): 
    loss = train()
    auc, ap = test()
    if epoch % 10 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}, AP: {ap:.4f}')

Epoch 010, Loss: 3.0437, AUC: 0.7207, AP: 0.7286
Epoch 020, Loss: 1.7090, AUC: 0.6511, AP: 0.6726
